In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json
import chromadb

c:\Users\shche\Desktop\chroma_db\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from docx import Document


chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name='vectors')

documents = []
folder_path = "documents"
for filename in os.listdir(folder_path):
    if filename.endswith('.docx'):
        file_path = os.path.join(folder_path,filename)
        doc = Document(file_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs ])
        documents.append(text)
ids = []
for i in range(len(documents)):
    ids.append(f'doc{i}')     
 
collection.add(
     documents=documents,
     ids = ids
)

import json

# Получение всех документов из ChromaDB
all_documents = collection.get(ids=ids)["documents"]

# Сохранение в JSON-файл
passages = [{"id": f"doc{i+1}", "text": doc} for i, doc in enumerate(all_documents)]
with open("passages.json", "w", encoding="utf-8") as f:
    json.dump(passages, f, ensure_ascii=False, indent=4)

In [11]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
# Загрузка документов из JSON-файла
def load_passages_from_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return [item["text"] for item in data]  # Предполагается, что тексты хранятся в поле "text"

passages = 'passages.json'

# Путь к вашему JSON-файлу
json_path = "passages.json"
passages = load_passages_from_json(json_path)

passage_embeddings = embedder.encode(passages)
print(passage_embeddings.shape)

dimension = passage_embeddings.shape[1]
dimension
# index = faiss.IndexFlatL2(dimension)
# index.add(passage_embeddings)

# # Функция для поиска документов
# def retrieve_documents(query, k=3):
#     query_embedding = embedder.encode([query])
#     distances, indices = index.search(query_embedding, k)
#     return [passages[i] for i in indices[0]]

(1, 384)


384

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer


# Загрузка генеративной модели (например, T5)
generator = T5ForConditionalGeneration.from_pretrained("t5-small")
generator_tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Функция для генерации ответа
def generate_answer(query, context):
    input_text = f"question: {query} context: {context}"
    input_ids = generator_tokenizer(input_text, return_tensors="pt").input_ids
    outputs = generator.generate(input_ids)
    return generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

c:\Users\shche\Desktop\chroma_db\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shche\.cache\huggingface\hub\models--facebook--bart-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [23]:
query = "Что такое SLAM?"

retrieved_docs = retrieve_documents(query)
context = " ".join(retrieved_docs)

answer = generate_answer(query,context)
print(f"Ответ:{answer} ")

IndexError: index out of range in self